[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Kaggle-runa/MameLand_vol3/blob/main/src/notebook/00_%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0.ipynb)

In [1]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 25.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,513 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InR

In [2]:
# 必要なライブラリをimport
import dataclasses
from tqdm import tqdm
import time
import io
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [7]:
# netkeibaにログインするためのメールアドレスとパスワードを設定して下さい
# これらを設定しない場合でもスピード指数など以外は取得できます

mail = "311.1999.kenta@gmail.com"
password = "ano12pmo"

### レースIDを取得するコード

In [4]:
# ユーザーエージェントの指定(自分のブラウザのユーザーエージェントを指定してください)
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}

In [5]:
# from_とto_で指定した期間の競馬の開催日を取得する関数
def get_kaisai_date(from_: str, to_: str):
    """
    yyyy-mm-ddの形式でfrom_とto_を指定すると、間の開催日程一覧が返ってくる関数。
    """

    # 日付範囲を生成
    date_range = pd.date_range(start=from_, end=to_, freq="D")
    kaisai_date_list = []
    seen_year_month = set()

    for date in tqdm(date_range, total=len(date_range)):
        year_month = (date.year, date.month)
        if year_month not in seen_year_month:
            seen_year_month.add(year_month)
            query = [
                'year=' + str(date.year),
                'month=' + str(date.month),
            ]
            url = "https://race.netkeiba.com/top/calendar.html?" + '&'.join(query)
            try:
                # ユーザーエージェントを設定してリクエストを送信
                response = requests.get(url, headers=headers)
                response.raise_for_status()  # ステータスコードがエラーの場合は例外を発生させる
                time.sleep(1)  # リクエスト間隔を空ける

                soup = BeautifulSoup(response.content, "lxml")
                table = soup.find('table', class_='Calendar_Table')
                if not table:
                    print(f"No calendar data found for {year_month}.")
                    continue
                a_list = table.find_all('a', href=True)
                for a in a_list:
                    match = re.search(r'kaisai_date=(\d+)', a['href'])
                    if match:
                        kaisai_date = match.group(1)
                        kaisai_date_list.append(kaisai_date)
            except Exception as e:
                print(f"Error fetching data for {year_month}: {e}")
                continue

    # 取得した開催日をフィルタリングして指定範囲に含まれる日付のみを返す
    kaisai_date_list = [d for d in kaisai_date_list if from_.replace("-", "") <= d <= to_.replace("-", "")]

    return kaisai_date_list

In [6]:
# get_kaisai_dateの期間中のrace_idの一覧を取得する関数
def get_race_id_list(kaisai_date_list: list):
    """
    開催日をyyyymmddの文字列形式でリストで入れると、レースid一覧が返ってくる関数
    """
    race_id_list = []

    # Chromeドライバの設定
    options = Options()
    options.add_argument("--headless")  # ヘッドレスモード
    options.add_argument('--disable-dev-shm-usage')  # メモリ関連の制限を回避
    options.add_argument("--no-sandbox")  # サンドボックスを無効化
    options.add_argument("--window-size=1920,1080")

    # Use webdriver.Chrome with the specified options
    driver = webdriver.Chrome(options=options)

    for kaisai_date in tqdm(kaisai_date_list):
        try:
            query = [
                'kaisai_date=' + str(kaisai_date)
            ]
            url = 'https://race.netkeiba.com/top/race_list.html' + '?' + '&'.join(query)
            print('scraping: {}'.format(url))
            driver.get(url)
            try:
                time.sleep(1)  # Wait for the page to load
                a_list = driver.find_element(By.CLASS_NAME, 'RaceList_Box').find_elements(By.TAG_NAME, 'a')
            except Exception:  # If not loaded, wait more
                print('waiting more 10 seconds')
                time.sleep(10)
                a_list = driver.find_element(By.CLASS_NAME, 'RaceList_Box').find_elements(By.TAG_NAME, 'a')
            for a in a_list:
                race_id = re.findall(r'(?<=result.html\?race_id=)\d+', a.get_attribute('href'))
                if len(race_id) > 0:
                    race_id_list.append(race_id[0])
        except Exception as e:
            print(f"Error: {e}")
            break
    driver.quit()
    return race_id_list

# 使用例
from_date = "2024-10-01"
to_date = "2024-10-31"

# 開催日一覧を取得
kaisai_date_list = get_kaisai_date(from_date, to_date)
print("開催日:", kaisai_date_list)

# レースID一覧を取得
race_id_list = get_race_id_list(kaisai_date_list)
print("レースID:", race_id_list)

  0%|          | 0/31 [00:00<?, ?it/s]

開催日: ['20241005', '20241006', '20241012', '20241013', '20241014', '20241019', '20241020', '20241026', '20241027']


  0%|          | 0/9 [00:00<?, ?it/s]

scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20241005
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20241006
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20241012
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20241013
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20241014
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20241019
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20241020
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20241026
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20241027
レースID: ['202405040101', '202405040102', '202405040103', '202405040104', '202405040105', '202405040106', '202405040107', '202405040108', '202405040109', '202405040110', '202405040111', '202405040112', '202408050101', '202408050102', '202408050103', '202408050104', '202408050105', '202408050106', '202408050107', '202

## レース結果を取得

In [8]:
# レース結果を取得するためのコード
def fetch_race_results(race_ids):

    def get_html_content(url):
        response = requests.get(url, headers=headers)
        response.encoding = "EUC-JP"
        return response.text

    def parse_race_data(html, race_id):
        # HTMLから最初のテーブルを取得し、列名の半角スペースを除去
        df = pd.read_html(io.StringIO(html))[0]
        df.columns = df.columns.str.replace(' ', '')

        # HTMLからレース情報部分のテキストを取得
        soup = BeautifulSoup(html, "html.parser")
        race_info_text = soup.find(class_="racedata fc").find("span").contents[0]

         # レース情報をDataFrameに追加
        df["race_title"] = [soup.find(class_="racedata fc").find("h1").text] * len(df)
        df["race_type"] = [race_info_text[0]] * len(df)
        df["race_turn"] = [race_info_text[1]] * len(df)
        df["course_len"] = [re.findall(r'\d{4}', race_info_text)[0]] * len(df)
        df["weather"] = [re.findall(r'天候\s*:\s*([^\/]+)', race_info_text.replace("\xa0", ""))[0]] * len(df)
        df["ground_condition"] = [re.findall(r'良|稍重|重|不良', race_info_text)[0]] * len(df)
        df["year"] = [re.findall(r'(\d{4})', soup.find(class_="smalltxt").contents[0])[0]] * len(df)
        df["date"] = [re.findall(r'(\d{1,2}月\d{1,2}日)', soup.find(class_="smalltxt").contents[0])[0]] * len(df)
        df["location"] = [re.findall(r'\d+回(..)', soup.find(class_="smalltxt").contents[0])[0]] * len(df)

        # 馬ID、騎手IDをDataFrameに追加
        df["horse_id"] = [re.findall(r"\d+", a["href"])[0] for a in soup.find("table", summary="レース結果").find_all("a", href=re.compile("^/horse"))]
        df["jockey_id"] = [re.findall(r"\d+", a["href"])[0] for a in soup.find("table", summary="レース結果").find_all("a", href=re.compile("^/jockey"))]
        df["race_id"] = [race_id] * len(df)

        return df

    race_data_dict = {} # レースデータを格納する辞書

    for race_id in tqdm(race_ids):
        time.sleep(1) # スクレイピングのマナーとして1秒待機
        print(f'Fetching data for race_id: {race_id}')

        try:
            url = f"https://db.netkeiba.com/race/{race_id}"
            html_content = get_html_content(url) # HTMLコンテンツを取得
            race_df = parse_race_data(html_content, race_id) # HTMLを解析してDataFrameを作成
            race_data_dict[race_id] = race_df # 辞書にDataFrameを追加
        except (IndexError, AttributeError):
            continue # 辞書にDataFrameを追加
        except Exception as e:
            print(f'Error occurred: {e}') # その他のエラーが発生した場合はエラーメッセージを表示
            break

    combined_results_df = pd.concat(race_data_dict.values())
    return combined_results_df

In [9]:
# レース結果を取得
race_results = fetch_race_results(race_id_list)

# 日付列を結合して新しい列を作成し、不要な列を削除
race_results['event_date'] = pd.to_datetime(race_results['year'].astype(str) + '年' + race_results['date'], format='%Y年%m月%d日').dt.strftime('%Y-%m-%d')
race_results = race_results.drop(['year', 'date'], axis=1)

# 列名の修正と並び替えを同時に実行
new_order = [
    'race_id', 'event_date', 'location', 'race_title', 'race_type', 'race_turn',
    'course_len', 'weather', 'ground_condition', 'finish_position', 'frame_number',
    'horse_number', 'horse_id', 'horse_name', 'sex_age', 'carried_weight',
    'jockey_id', 'jockey', 'time', 'difference', 'odds', 'popularity',
    'horse_weight', 'trainer'
]

race_results.columns = [
    'finish_position', 'frame_number', 'horse_number', 'horse_name', 'sex_age', 'carried_weight',
    'jockey', 'time', 'difference', 'odds', 'popularity', 'horse_weight', 'trainer', 'race_title',
    'race_type', 'race_turn', 'course_len', 'weather', 'ground_condition', 'location', 'horse_id',
    'jockey_id', 'race_id', 'event_date'
]

# 列を並び替え
race_results = race_results[new_order]

# データの保存
race_results.to_csv('race_results.csv',index=None)

  0%|          | 0/10 [00:00<?, ?it/s]

Fetching data for race_id: 202405040101
Fetching data for race_id: 202405040102
Fetching data for race_id: 202405040103
Fetching data for race_id: 202405040104
Fetching data for race_id: 202405040105
Fetching data for race_id: 202405040106
Fetching data for race_id: 202405040107
Fetching data for race_id: 202405040108
Fetching data for race_id: 202405040109
Fetching data for race_id: 202405040110


## 競走馬の過去の成績を取得

In [26]:
def login_and_get_session(mail, password):
    session = requests.Session()
    login_data = {
        'login_id': mail,
        'pswd': password
    }
    login_url = "https://regist.netkeiba.com/account/?pid=login&action=auth"
    response = session.post(login_url, data=login_data)

    print(response.url)

    if response.url != login_url:
        print("ログイン成功")
    else:
        print("ログイン失敗")
        raise Exception("ログインに失敗しました")

    return session

def fetch_horse_results(horse_id_list, session):
    horse_results = {}
    for horse_id in tqdm(horse_id_list):
        time.sleep(1)
        try:
            url = f'https://db.netkeiba.com/horse/{horse_id}'
            response = session.get(url, headers=headers)
            if response.status_code == 200:
                df_list = pd.read_html(response.content, encoding='euc-jp')
                df = df_list[2]
                if df.columns[0] == '受賞歴':
                    df = df_list[4]
                df['horse_id'] = [horse_id] * len(df)
                horse_results[horse_id] = df
            else:
                print(f"Failed to retrieve data for horse_id: {horse_id}")
        except IndexError:
            continue
        except Exception as e:
            print(e)
            break

    if horse_results:
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])
    else:
        horse_results_df = pd.DataFrame()

    return horse_results_df

In [29]:
# レースに参加した馬のidを取得
horse_id_list = race_results['horse_id'].unique()

# ログインしてセッションを取得
session = login_and_get_session(mail, password)

# 過去成績データをスクレイピング
horse_results_df = fetch_horse_results(horse_id_list, session)

# 不要な列を削除
horse_results_df = horse_results_df.drop(['映 像', '厩舎 ｺﾒﾝﾄ', '備考'],axis=1)

# 列名の変更
horse_results_df.columns = [
    'date', 'venue', 'weather', 'race_number', 'race_name', 'num_horses', 'frame_number', 'horse_number',
    'odds', 'popularity', 'finish_position', 'jockey', 'carried_weight', 'distance', 'track_condition',
    'track_index', 'time', 'difference', 'time_index', 'passing_order', 'pace', 'last_3f', 'horse_weight',
    'winner_or_2nd', 'prize_money', 'horse_id'
]

# 日付の変換(yyyy-mm-dd)に
horse_results_df['date'] = pd.to_datetime(horse_results_df['date'], format='%Y/%m/%d')

# yyyy-mm-dd形式に変換
horse_results_df['date'] = horse_results_df['date'].dt.strftime('%Y-%m-%d')

horse_results_df.to_csv('horse_results.csv',index=None)

https://regist.netkeiba.com/account/?pid=redirect&url=https%3A%2F%2Fwww.netkeiba.com%2F
ログイン成功


  0%|          | 0/2 [00:00<?, ?it/s]

           日付    開催 天 気  R   レース名  映 像  頭 数  枠 番  馬 番  オ ッ ズ  ...   着差  \
0  2024/10/26  5京都7   曇  8  萩S(L)  NaN    6    4    4   12.4  ...  1.9   
1  2024/10/05  4東京1  小雨  1  2歳未勝利  NaN    9    7    7   14.8  ... -0.2   
2  2024/09/07  3中京1   晴  5   2歳新馬  NaN    8    2    2    3.8  ...  1.0   

   ﾀｲﾑ 指数     通過        ペース    上り      馬体重  厩舎 ｺﾒﾝﾄ   備考  勝ち馬 (2着馬)     賞金  
0      71    6-6  37.2-33.7  34.7   480(0)      NaN  アオル     テリオスララ    NaN  
1      83    9-9  34.2-35.3  34.6  480(-6)      NaN  出遅れ  (カネラフィーナ)  550.0  
2      61  3-3-4  37.8-33.9  34.5   486(0)      NaN  出遅れ   タイセイカレント  180.0  

[3 rows x 28 columns]
           日付    開催 天 気  R   レース名  映 像  頭 数  枠 番  馬 番  オ ッ ズ  ...   着差  \
0  2024/11/02  5東京1  小雨  4  2歳未勝利  NaN    8    8    8    1.7  ...  0.0   
1  2024/10/05  4東京1  小雨  1  2歳未勝利  NaN    9    6    6    1.5  ...  0.2   
2  2024/08/03  2新潟3   晴  4   2歳新馬  NaN   11    2    2    2.2  ...  0.4   

   ﾀｲﾑ 指数     通過        ペース    上り      馬体重  厩舎 ｺﾒﾝﾄ  備考  勝ち馬 (2着馬)     賞金  

## スピード指数を取得

In [30]:
def fetch_index(original_race_id_list):
    # ID変換の対応を保持
    id_mapping = {int(str(id)[2:]): id for id in original_race_id_list}
    converted_race_id_list = list(id_mapping.keys())

    all_index_list = []

    for race_id in tqdm(converted_race_id_list):
        time.sleep(1)
        i = 0
        index_list = []

        url = 'https://jiro8.sakura.ne.jp/index2.php?code=' + str(race_id)
        url_html = requests.get(url, headers=headers)
        html = BeautifulSoup(url_html.content, 'html.parser')
        RaceTable01 = html.findAll('table', {'class':'c1'})[0]

        for RaceTable01_tr in RaceTable01.findAll('tr'):
            if i >= 1:
                if len(RaceTable01_tr.findAll('td')) > 7:
                    uma_ban = RaceTable01_tr.findAll('td')[1].get_text()
                    uma_ban = int(uma_ban)
                    zensou = RaceTable01_tr.findAll('td')[8]

                    speed_index = 0
                    rasing_index = 0
                    pace_index = 0
                    leading_index = 0

                    if len(zensou.findAll('span', {'class':'sn22'})) != 0:
                        speed_index = zensou.findAll('span', {'class':'sn22'})[0].get_text()
                        speed_index = float(speed_index)

                        rasing_index = zensou.findAll('span', {'class':'sn22'})[1].get_text()
                        rasing_index = float(rasing_index)

                        pace_index = zensou.findAll('span', {'class':'sn22'})[2].get_text()
                        pace_index = float(pace_index)

                        leading_index = zensou.findAll('span', {'class':'sn22'})[3].get_text()
                        leading_index = float(leading_index)

                    temp_list = [id_mapping[race_id], uma_ban, speed_index, rasing_index, pace_index, leading_index]
                    index_list.append(temp_list)
            i += 1

        df_index = pd.DataFrame(index_list)
        df_index = df_index.set_axis(['race_id', 'uma_ban', 'speed_index', 'rasing_index', 'pace_index', 'leading_index'], axis=1)
        all_index_list.append(df_index)

    all_race_df = pd.concat(all_index_list, ignore_index=True)
    return all_race_df

In [31]:
# 先ほど取得したレースIDでは存在しないrace_idもあるため、race_resultsにある存在するrace_idを持ってくる
unique_race_id_list = race_results['race_id'].unique()

# スピード指数の取得
speed_results = fetch_index(unique_race_id_list)

speed_results.to_csv('speed_results.csv',index=None)

  0%|          | 0/10 [00:00<?, ?it/s]

## 過去の払戻金を取得

In [34]:
def fetch_return(race_id_list):

    return_tables = {}
    for race_id in tqdm(race_id_list):
        time.sleep(1)
        try:
            url = "https://db.netkeiba.com/race/" + race_id
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # ステータスコード400/500系エラーをキャッチ

            soup = BeautifulSoup(response.content, 'lxml')
            html = str(soup).replace('<br />', 'br')  # HTMLを文字列として処理し改行タグを置換

            dfs = pd.read_html(html)
            df = pd.concat([dfs[1], dfs[2]])

            df['race_id'] = [race_id] * len(df)
            return_tables[race_id] = df

        except requests.exceptions.RequestException as e:
            print(f"Request error for race_id {race_id}: {e}")
            continue
        except IndexError:
            print(f"IndexError for race_id {race_id}: Tables not found.")
            continue
        except AttributeError:
            print(f"AttributeError for race_id {race_id}: Possibly an invalid race ID.")
            continue
        except Exception as e:
            print(f"Unexpected error for race_id {race_id}: {e}")
            break

    # pd.DataFrame型にして一つのデータにまとめる
    if return_tables:
        return_tables_df = pd.concat([return_tables[key] for key in return_tables])
        return return_tables_df
    else:
        print("No valid race data was fetched.")
        return pd.DataFrame()

In [35]:
# 先ほど取得したレースIDでは存在しないrace_idもあるため、race_resultsにある存在するrace_idを持ってくる
unique_race_id_list = race_results['race_id'].unique()

# 払い戻し表データを取得
results = fetch_return(unique_race_id_list)

# 列名を指定された名前に変更
results.columns = ['baken_types', 'horse_number', 'refund', 'popularity', 'race_id']

# race_id列を最初の列に移動
results = results[['race_id', 'baken_types', 'horse_number', 'refund', 'popularity']]

results.to_csv('pay_results.csv',index=None)

  0%|          | 0/10 [00:00<?, ?it/s]

<ipython-input-34-c88dda99b0ae>:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
<ipython-input-34-c88dda99b0ae>:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
<ipython-input-34-c88dda99b0ae>:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
<ipython-input-34-c88dda99b0ae>:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(html)
<ipython-input-34-c88dda99b0ae>:14: FutureWarning: Passing literal html 